### Load packages

In [1]:
import pandas as pd

### Load data

In [2]:
#change paths to the nice paths i used in latest projects. Not super needed here since we dont rely a lot on relative paths

In [3]:
#path_data = 'G:/My Drive/ML-AI/Projects/eurobasket_predictor/data/'

path_data = 'C:/Users/Carlos Ivan/Documents/Projects/eurobasket_predictor/data/'

In [4]:
file='Matches.xlsx'

In [5]:
matches = pd.read_excel(path_data + file)


In [6]:
matches.head(2)

,date_match,team_1,score_team_1,team_2,score_team_2
0,801,Estonia,68,Lithuania,89
1,802,Iceland,61,Italy,87


In [7]:
classes = pd.read_excel(path_data + 'Teams Ranking Classes.xlsx')


In [8]:
classes.head(2)

,Team,Class
0,Belgium,Normal
1,Bosnia and Herzegovina,Normal


In [9]:
matches.shape

(67, 5)

### Duplicate rows, make team and opponent cols 
Id of a row will be the date of the match, and the team

In [11]:
# Before calculating stats, transform the table to columns:
#date_match, team, opponent, score_team, score_opponent (will have twice the number of rows)

In [12]:
matches.head(2)

,date_match,team_1,score_team_1,team_2,score_team_2
0,801,Estonia,68,Lithuania,89
1,802,Iceland,61,Italy,87


In [13]:
def get_df_two_rows_per_game(a_df):
    df=a_df.copy()
    two_rows_per_game = pd.DataFrame(columns=['date_match','team','opponent','score_team','score_opponent'])
    for a_match in range(len(df)):
        # Adding team 1
        new_row = {"date_match": df.loc[a_match, 'date_match'], "team": df.loc[a_match, 'team_1'],
                   "opponent": df.loc[a_match, 'team_2'],"score_team": df.loc[a_match, 'score_team_1'],
                   "score_opponent": df.loc[a_match, 'score_team_2']}
        two_rows_per_game = pd.concat([two_rows_per_game, pd.DataFrame([new_row])], ignore_index=True)

        # Adding team 2
        new_row = {"date_match": df.loc[a_match, 'date_match'], "team": df.loc[a_match, 'team_2'],
                   "opponent": df.loc[a_match, 'team_1'],"score_team": df.loc[a_match, 'score_team_2'],
                   "score_opponent": df.loc[a_match, 'score_team_1']}
        two_rows_per_game = pd.concat([two_rows_per_game, pd.DataFrame([new_row])], ignore_index=True)

    return two_rows_per_game

In [14]:
df_two_rows_per_game = get_df_two_rows_per_game(matches)

In [15]:
df_two_rows_per_game.head(3)

,date_match,team,opponent,score_team,score_opponent
0,801,Estonia,Lithuania,68,89
1,801,Lithuania,Estonia,89,68
2,802,Iceland,Italy,61,87


In [16]:
df_two_rows_per_game.shape

(134, 5)

### Add class

In [17]:
def add_classes(a_df, df_classes):
    df=a_df.copy()
    df["class_team"] = df["team"].map(df_classes.set_index("Team")["Class"])
    df["class_opponent"] = df["opponent"].map(df_classes.set_index("Team")["Class"])
    return df

In [18]:
df_with_classes = add_classes(df_two_rows_per_game, classes)
df_with_classes.head(3)

,date_match,team,opponent,score_team,score_opponent,class_team,class_opponent
0,801,Estonia,Lithuania,68,89,Normal,Top
1,801,Lithuania,Estonia,89,68,Top,Normal
2,802,Iceland,Italy,61,87,Normal,Top


### Add flag of won and points difference in the match

In [19]:
# Calculate stats of game

def add_winner_flags(a_df):
    df=a_df.copy()
    df['won'] = (df['score_team']>df['score_opponent']).astype(int)
    return df

def add_diff_points(a_df):
    df=a_df.copy()
    df['pts_diff']= df['score_team']-df['score_opponent']
    return df


In [20]:
df_with_classes = add_winner_flags(df_with_classes)
df_with_classes = add_diff_points(df_with_classes)

In [21]:
df_with_classes.head(5)

,date_match,team,opponent,score_team,score_opponent,class_team,class_opponent,won,pts_diff
0,801,Estonia,Lithuania,68,89,Normal,Top,0,-21
1,801,Lithuania,Estonia,89,68,Top,Normal,1,21
2,802,Iceland,Italy,61,87,Normal,Top,0,-26
3,802,Italy,Iceland,87,61,Top,Normal,1,26
4,803,Iceland,Poland,90,92,Normal,Normal,0,-2


### Aggregate per team

In [22]:
aggregated_df_raw = df_with_classes.groupby(['team',
                              'class_team',
                              'class_opponent']).agg({'won':['mean','count'],
                                                      'pts_diff': ['mean','count']}).reset_index()

In [23]:
aggregated_df_raw.columns = [
    "_".join([str(c) for c in col if c]) if isinstance(col, tuple) else col
    for col in aggregated_df_raw.columns
]

In [24]:
aggregated_df_raw.head(5)

,team,class_team,class_opponent,won_mean,won_count,pts_diff_mean,pts_diff_count
0,Argentina,Normal,Normal,1.0,1,14.0,1
1,Argentina,Normal,Top,0.0,1,-12.0,1
2,Belgium,Normal,Normal,0.2,5,-15.6,5
3,Belgium,Normal,Top,0.0,1,-12.0,1
4,Bosnia and Herzegovina,Normal,Normal,0.5,4,4.25,4


### Turn to wide

Have all info of a particular team in a single record

In [25]:
wide = (
    aggregated_df_raw.pivot_table(
        index=["team","class_team"] ,                               # rows
        columns=[ "class_opponent"],    # spread across columns
        values=["won_mean",  "pts_diff_mean", "won_count"]
    )
)

In [26]:
wide.columns = [
    f"{val}_{ct}" for val, ct in wide.columns
]
wide = wide.reset_index()
wide = wide.rename(columns={'won_count_Normal': 'count_Normal', 'won_count_Top': 'count_Top'})

#Reorder columns
wide= wide[['team', 'class_team', 'count_Top', 'count_Normal',	'won_mean_Top', 'won_mean_Normal', 'pts_diff_mean_Top', 'pts_diff_mean_Normal']]

In [27]:
wide.head(5)

,team,class_team,count_Top,count_Normal,won_mean_Top,won_mean_Normal,pts_diff_mean_Top,pts_diff_mean_Normal
0,Argentina,Normal,1.0,1.0,0.0,1.000000,-12.0,14.0
1,Belgium,Normal,1.0,5.0,0.0,0.200000,-12.0,-15.6
2,Bosnia and Herzegovina,Normal,1.0,4.0,0.0,0.500000,-37.0,4.25
3,Cyprus,Normal,1.0,1.0,0.0,0.000000,-65.0,-40.0
4,Czechia,Normal,3.0,3.0,0.0,0.666667,-19.0,9.333333


### Drop argentina and spain b

In [28]:
wide=wide[~wide["team"].isin(["Argentina", "Spain B"])].reset_index(drop=True)


In [29]:
wide.head(5)

,team,class_team,count_Top,count_Normal,won_mean_Top,won_mean_Normal,pts_diff_mean_Top,pts_diff_mean_Normal
0,Belgium,Normal,1.0,5.0,0.0,0.200000,-12.0,-15.6
1,Bosnia and Herzegovina,Normal,1.0,4.0,0.0,0.500000,-37.0,4.25
2,Cyprus,Normal,1.0,1.0,0.0,0.000000,-65.0,-40.0
3,Czechia,Normal,3.0,3.0,0.0,0.666667,-19.0,9.333333
4,Estonia,Normal,1.0,4.0,0.0,0.750000,-21.0,2.0


### Include a row of averages (will be handy)

In [30]:
means = wide.drop(columns='team').groupby('class_team').mean().reset_index()
means

,class_team,count_Top,count_Normal,won_mean_Top,won_mean_Normal,pts_diff_mean_Top,pts_diff_mean_Normal
0,Normal,1.5,3.785714,0.166667,0.463095,-17.277778,-1.328571
1,Top,4.0,2.375000,0.515000,0.895833,0.74,13.96875


In [31]:
means['team']='Average'

In [32]:
wide = pd.concat([wide, means]).reset_index(drop=True)

### Impute data

For win percentage:
* If a top team did not play against any normal team (just vs top ones), we assume they played 1 game vs normal and they won
* If a normal team did not play against any normal team, we assume they played 1 game vs normal and asign the average sliced
* If any team did not play against any top team, we assume they played 1 game vs top and they lost

For points difference:
* We just assigned the average sliced

In [33]:
wide

,team,class_team,count_Top,count_Normal,won_mean_Top,won_mean_Normal,pts_diff_mean_Top,pts_diff_mean_Normal
0,Belgium,Normal,1.0,5.000000,0.000000,0.200000,-12.0,-15.6
1,Bosnia and Herzegovina,Normal,1.0,4.000000,0.000000,0.500000,-37.0,4.25
2,Cyprus,Normal,1.0,1.000000,0.000000,0.000000,-65.0,-40.0
3,Czechia,Normal,3.0,3.000000,0.000000,0.666667,-19.0,9.333333
4,Estonia,Normal,1.0,4.000000,0.000000,0.750000,-21.0,2.0
5,Finland,Normal,NaN,4.000000,NaN,1.000000,NaN,22.25
6,France,Top,3.0,2.000000,1.000000,1.000000,9.333333,6.5
7,Georgia,Normal,1.0,5.000000,0.000000,0.000000,-23.0,-10.6
8,Germany,Top,6.0,NaN,0.833333,NaN,5.666667,NaN
9,Great Britain,Normal,2.0,3.000000,0.000000,0.333333,-9.5,-0.666667


In [34]:
wide_imputed = wide.copy()

In [35]:
wide_imputed.loc[wide['won_mean_Top'].isna(), 'won_mean_Top'] = 0
wide_imputed.loc[wide['won_mean_Top'].isna(), 'count_Top'] = 1

wide_imputed.loc[(wide['won_mean_Normal'].isna()) & (wide['class_team']=='Top'), 'won_mean_Normal'] = 1
wide_imputed.loc[(wide['won_mean_Normal'].isna()) & (wide['class_team']=='Top') , 'count_Normal'] = 1

wide_imputed.loc[(wide['won_mean_Normal'].isna()) & (wide['class_team']=='Normal'), 'won_mean_Normal'] = wide.loc[(wide['team']=='Average')&(wide['class_team']=='Normal'), 'won_mean_Normal'].values[0]
wide_imputed.loc[(wide['won_mean_Normal'].isna()) & (wide['class_team']=='Normal') , 'count_Normal'] = 1


In [36]:

#For points difference:
wide_imputed.loc[(wide['won_mean_Top'].isna()) & (wide['class_team']=='Top'), 'pts_diff_mean_Top'] = wide.loc[(wide['team']=='Average')&(wide['class_team']=='Top'), 'pts_diff_mean_Top'].values[0]
wide_imputed.loc[(wide['won_mean_Top'].isna()) & (wide['class_team']=='Normal'), 'pts_diff_mean_Top'] = wide.loc[(wide['team']=='Average')&(wide['class_team']=='Normal'), 'pts_diff_mean_Top'].values[0]

wide_imputed.loc[(wide['won_mean_Normal'].isna()) & (wide['class_team']=='Top'), 'pts_diff_mean_Normal'] = wide.loc[(wide['team']=='Average')&(wide['class_team']=='Top'), 'pts_diff_mean_Normal'].values[0]
wide_imputed.loc[(wide['won_mean_Normal'].isna()) & (wide['class_team']=='Normal'), 'pts_diff_mean_Normal'] = wide.loc[(wide['team']=='Average')&(wide['class_team']=='Normal'), 'pts_diff_mean_Normal'].values[0]


In [37]:
wide_imputed

,team,class_team,count_Top,count_Normal,won_mean_Top,won_mean_Normal,pts_diff_mean_Top,pts_diff_mean_Normal
0,Belgium,Normal,1.0,5.000000,0.000000,0.200000,-12.0,-15.6
1,Bosnia and Herzegovina,Normal,1.0,4.000000,0.000000,0.500000,-37.0,4.25
2,Cyprus,Normal,1.0,1.000000,0.000000,0.000000,-65.0,-40.0
3,Czechia,Normal,3.0,3.000000,0.000000,0.666667,-19.0,9.333333
4,Estonia,Normal,1.0,4.000000,0.000000,0.750000,-21.0,2.0
5,Finland,Normal,1.0,4.000000,0.000000,1.000000,-17.277778,22.25
6,France,Top,3.0,2.000000,1.000000,1.000000,9.333333,6.5
7,Georgia,Normal,1.0,5.000000,0.000000,0.000000,-23.0,-10.6
8,Germany,Top,6.0,1.000000,0.833333,1.000000,5.666667,13.96875
9,Great Britain,Normal,2.0,3.000000,0.000000,0.333333,-9.5,-0.666667


### Compare teams using weighted values

In [38]:
#Brief description here of the logic used

In [80]:
def compare_weighted_stats(team1, team2):
    class_team_1 = wide_imputed.loc[wide_imputed['team']==team1,'class_team'].values[0]
    class_team_2 = wide_imputed.loc[wide_imputed['team']==team2,'class_team'].values[0]
    count_Top_team_1 = wide_imputed.loc[wide_imputed['team']==team1,'count_Top'].values[0]
    count_Top_team_2 = wide_imputed.loc[wide_imputed['team']==team2,'count_Top'].values[0]
    count_Normal_team_1 = wide_imputed.loc[wide_imputed['team']==team1,'count_Normal'].values[0]
    count_Normal_team_2 = wide_imputed.loc[wide_imputed['team']==team2,'count_Normal'].values[0]
    won_mean_Top_team_1 = wide_imputed.loc[wide_imputed['team']==team1,'won_mean_Top'].values[0]
    won_mean_Top_team_2 = wide_imputed.loc[wide_imputed['team']==team2,'won_mean_Top'].values[0]
    won_mean_Normal_team_1 = wide_imputed.loc[wide_imputed['team']==team1,'won_mean_Normal'].values[0]
    won_mean_Normal_team_2 = wide_imputed.loc[wide_imputed['team']==team2,'won_mean_Normal'].values[0]
    pts_diff_mean_Top_team_1 = wide_imputed.loc[wide_imputed['team']==team1,'pts_diff_mean_Top'].values[0]
    pts_diff_mean_Top_team_2 = wide_imputed.loc[wide_imputed['team']==team2,'pts_diff_mean_Top'].values[0]
    pts_diff_mean_Normal_team_1 = wide_imputed.loc[wide_imputed['team']==team1,'pts_diff_mean_Normal'].values[0]
    pts_diff_mean_Normal_team_2 = wide_imputed.loc[wide_imputed['team']==team2,'pts_diff_mean_Normal'].values[0]

    avg_sliced_won_team_1_Top = wide_imputed.loc[(wide_imputed['team']=='Average')&(wide_imputed['class_team']==class_team_1), 'won_mean_Top'].values[0]
    avg_sliced_won_team_2_Top = wide_imputed.loc[(wide_imputed['team']=='Average')&(wide_imputed['class_team']==class_team_2), 'won_mean_Top'].values[0]
    avg_sliced_won_team_1_Normal = wide_imputed.loc[(wide_imputed['team']=='Average')&(wide_imputed['class_team']==class_team_1), 'won_mean_Normal'].values[0]
    avg_sliced_won_team_2_Normal = wide_imputed.loc[(wide_imputed['team']=='Average')&(wide_imputed['class_team']==class_team_2), 'won_mean_Normal'].values[0]

    avg_sliced_pts_team_1_Top = wide_imputed.loc[(wide_imputed['team']=='Average')&(wide_imputed['class_team']==class_team_1), 'pts_diff_mean_Top'].values[0]
    avg_sliced_pts_team_2_Top = wide_imputed.loc[(wide_imputed['team']=='Average')&(wide_imputed['class_team']==class_team_2), 'pts_diff_mean_Top'].values[0]
    avg_sliced_pts_team_1_Normal = wide_imputed.loc[(wide_imputed['team']=='Average')&(wide_imputed['class_team']==class_team_1), 'pts_diff_mean_Normal'].values[0]
    avg_sliced_pts_team_2_Normal = wide_imputed.loc[(wide_imputed['team']=='Average')&(wide_imputed['class_team']==class_team_2), 'pts_diff_mean_Normal'].values[0]

 
    ###### Start with team 2 stats #######
    #Calculate win pct and points diff for Team 2:
    if class_team_1== 'Normal':  #stats to use depend on type of oponent
        factor_1= won_mean_Normal_team_2
        factor_1_pts=pts_diff_mean_Normal_team_2
        if count_Normal_team_2>1:
            if count_Top_team_2>1:  #Case 1: enough samples from both slices
                multiplier= count_Normal_team_2
                factor_2= won_mean_Top_team_2
                win_pct_team_2=(factor_1*multiplier + factor_2)/(multiplier+1)
                factor_2_pts=pts_diff_mean_Top_team_2
                pts_diff_team_2=(factor_1_pts*multiplier + factor_2_pts)/(multiplier+1)
            if count_Top_team_2==1: #Case 2: enough samples of the one of interest, but few of the other one
                multiplier= count_Normal_team_2
                factor_2= (won_mean_Top_team_2+avg_sliced_won_team_2_Top )/2
                win_pct_team_2=(factor_1*multiplier + factor_2)/(multiplier+1)
                factor_2_pts=(pts_diff_mean_Top_team_2+avg_sliced_pts_team_2_Top)/2
                pts_diff_team_2=(factor_1_pts*multiplier + factor_2_pts)/(multiplier+1)
        if count_Normal_team_2==1:
            if count_Top_team_2>1:  #Case 3: few samples of the one of interest but enough samples from the other one
                factor_2=avg_sliced_won_team_2_Normal
                factor_3=won_mean_Top_team_2
                win_pct_team_2=(factor_1 + factor_2 + factor_3)/3
                factor_2_pts=avg_sliced_pts_team_2_Normal
                factor_3_pts=pts_diff_mean_Top_team_2
                pts_diff_team_2=(factor_1_pts + factor_2_pts + factor_3_pts)/3
            if count_Top_team_2==1: #Case 4: few samples of both slices
                factor_2=avg_sliced_won_team_2_Normal
                factor_3=(won_mean_Top_team_2+avg_sliced_won_team_2_Top)/2
                win_pct_team_2=(factor_1 + factor_2 + factor_3)/3
                factor_2_pts=avg_sliced_pts_team_2_Normal
                factor_3_pts=(pts_diff_mean_Top_team_2+avg_sliced_pts_team_2_Top)/2
                pts_diff_team_2=(factor_1_pts + factor_2_pts + factor_3_pts)/3
    if class_team_1== 'Top': 
        factor_1=won_mean_Top_team_2
        factor_1_pts=pts_diff_mean_Top_team_2
        if count_Top_team_2>1:
            if count_Normal_team_2>1:  #Case 5: enough samples from both slices
                multiplier= count_Top_team_2
                factor_2=won_mean_Normal_team_2
                win_pct_team_2=(factor_1*multiplier + factor_2)/(multiplier+1)
                factor_2_pts= pts_diff_mean_Normal_team_2
                pts_diff_team_2= (factor_1_pts*multiplier + factor_2_pts)/(multiplier+1)
            if count_Normal_team_2==1: #Case 6: enough samples of the one of interest, but few of the other one
                multiplier= count_Top_team_2
                factor_2= (won_mean_Normal_team_2+avg_sliced_won_team_2_Normal )/2
                win_pct_team_2=(factor_1*multiplier + factor_2)/(multiplier+1)
                factor_2_pts= (pts_diff_mean_Normal_team_2+avg_sliced_pts_team_2_Normal )/2
                pts_diff_team_2 =(factor_1_pts*multiplier + factor_2_pts)/(multiplier+1)
        if count_Top_team_2==1:
            if count_Normal_team_2>1:  #Case 7: few samples of the one of interest but enough samples from the other one
                factor_2=avg_sliced_won_team_2_Top
                factor_3=won_mean_Normal_team_2
                win_pct_team_2=(factor_1 + factor_2 + factor_3)/3
                factor_2_pts= avg_sliced_pts_team_2_Top
                factor_3_pts= pts_diff_mean_Normal_team_2
                pts_diff_team_2= (factor_1_pts + factor_2_pts + factor_3_pts)/3
            if count_Normal_team_2==1: #Case 8: few samples of both slices
                factor_2=avg_sliced_won_team_2_Top
                factor_3=(won_mean_Normal_team_2+avg_sliced_won_team_2_Normal)/2
                win_pct_team_2=(factor_1 + factor_2 + factor_3)/3
                factor_2_pts= avg_sliced_pts_team_2_Top
                factor_3_pts= (pts_diff_mean_Normal_team_2+avg_sliced_pts_team_2_Normal)/2
                pts_diff_team_2= (factor_1_pts + factor_2_pts + factor_3_pts)/3

    ###### Now for team 1 #######
    #Calculate win pct and points diff for Team 1:
    if class_team_2== 'Normal': # stats to use depend on oponents class
        factor_1= won_mean_Normal_team_1
        factor_1_pts= pts_diff_mean_Normal_team_1
        if count_Normal_team_1>1:
            if count_Top_team_1>1:  #Case 1: enough samples from both slices
                multiplier= count_Normal_team_1
                factor_2= won_mean_Top_team_1
                win_pct_team_1=(factor_1*multiplier + factor_2)/(multiplier+1)
                factor_2_pts=pts_diff_mean_Top_team_1
                pts_diff_team_1=(factor_1_pts*multiplier + factor_2_pts)/(multiplier+1)
            if count_Top_team_1==1: #Case 2: enough samples of the one of interest, but few of the other one
                multiplier= count_Normal_team_1
                factor_2= (won_mean_Top_team_1+avg_sliced_won_team_1_Top )/2
                win_pct_team_1=(factor_1*multiplier + factor_2)/(multiplier+1)
                factor_2_pts=(pts_diff_mean_Top_team_1+avg_sliced_pts_team_1_Top)/2
                pts_diff_team_1=(factor_1_pts*multiplier + factor_2_pts)/(multiplier+1)
        if count_Normal_team_1==1:
            if count_Top_team_1>1:  #Case 3: few samples of the one of interest but enough samples from the other one
                factor_2=avg_sliced_won_team_1_Normal
                factor_3=won_mean_Top_team_1
                win_pct_team_1=(factor_1 + factor_2 + factor_3)/3
                factor_2_pts=avg_sliced_pts_team_1_Normal
                factor_3_pts=pts_diff_mean_Top_team_1
                pts_diff_team_1=(factor_1_pts + factor_2_pts + factor_3_pts)/3
            if count_Top_team_1==1: #Case 4: few samples of both slices
                factor_2=avg_sliced_won_team_1_Normal
                factor_3=(won_mean_Top_team_1+avg_sliced_won_team_1_Top)/2
                win_pct_team_1=(factor_1 + factor_2 + factor_3)/3
                factor_2_pts=avg_sliced_pts_team_1_Normal
                factor_3_pts=(pts_diff_mean_Top_team_1+avg_sliced_pts_team_1_Top)/2
                pts_diff_team_1=(factor_1_pts + factor_2_pts + factor_3_pts)/3
    if class_team_2== 'Top': 
        factor_1=won_mean_Top_team_1
        factor_1_pts=pts_diff_mean_Top_team_1
        if count_Top_team_1>1:
            if count_Normal_team_1>1:  #Case 5: enough samples from both slices
                multiplier= count_Top_team_1
                factor_2=won_mean_Normal_team_1
                win_pct_team_1=(factor_1*multiplier + factor_2)/(multiplier+1)
                factor_2_pts= pts_diff_mean_Normal_team_1
                pts_diff_team_1= (factor_1_pts*multiplier + factor_2_pts)/(multiplier+1)
            if count_Normal_team_1==1: #Case 6: enough samples of the one of interest, but few of the other one
                multiplier= count_Top_team_1
                factor_2= (won_mean_Normal_team_1+avg_sliced_won_team_1_Normal )/2
                win_pct_team_1=(factor_1*multiplier + factor_2)/(multiplier+1)
                factor_2_pts= (pts_diff_mean_Normal_team_1+avg_sliced_pts_team_1_Normal )/2
                pts_diff_team_1 =(factor_1_pts*multiplier + factor_2_pts)/(multiplier+1)
        if count_Top_team_1==1:
            if count_Normal_team_1>1:  #Case 7: few samples of the one of interest but enough samples from the other one
                factor_2=avg_sliced_won_team_1_Top
                factor_3=won_mean_Normal_team_1
                win_pct_team_1=(factor_1 + factor_2 + factor_3)/3
                factor_2_pts= avg_sliced_pts_team_1_Top
                factor_3_pts= pts_diff_mean_Normal_team_1
                pts_diff_team_1= (factor_1_pts + factor_2_pts + factor_3_pts)/3
            if count_Normal_team_1==1: #Case 8: few samples of both slices
                factor_2=avg_sliced_won_team_1_Top
                factor_3=(won_mean_Normal_team_1+avg_sliced_won_team_1_Normal)/2
                win_pct_team_1=(factor_1 + factor_2 + factor_3)/3
                factor_2_pts= avg_sliced_pts_team_1_Top
                factor_3_pts= (pts_diff_mean_Normal_team_1+avg_sliced_pts_team_1_Normal)/2
                pts_diff_team_1= (factor_1_pts + factor_2_pts + factor_3_pts)/3

    winner=''
    pts_difference=0
    if  win_pct_team_1>win_pct_team_2:
        winner=team1
        pts_difference= (pts_diff_team_1 - pts_diff_team_2)/2
    elif  win_pct_team_2>win_pct_team_1:
        winner=team2
        pts_difference= (pts_diff_team_2 - pts_diff_team_1)/2

    # return {team1:{'win_chances':float(win_pct_team_1),
    #                'pts_diff':float(pts_diff_team_1)},
    #         team2:{'win_chances':float(win_pct_team_2),
    #                'pts_diff':float(pts_diff_team_2)},
    #         'Suggestion':{'winner': winner  ,
    #                       'pts_difference': float(pts_difference)  }
    #         }

    return{'team_1': team1,
           'team_2': team2,
           't1_win_chances':float(win_pct_team_1),
           't2_win_chances':float(win_pct_team_2),
           't1_pts_diff':float(pts_diff_team_1),
           't2_pts_diff':float(pts_diff_team_2),
           'suggested_winner': winner,
           'pts_diff':float(pts_difference)
           }



In [81]:
compare_weighted_stats('Spain', 'Finland')

{'team_1': 'Spain',
 'team_2': 'Finland',
 't1_win_chances': 0.3333333333333333,
 't2_win_chances': 0.3888888888888889,
 't1_pts_diff': 1.4166666666666667,
 't2_pts_diff': -4.101851851851852,
 'suggested_winner': 'Finland',
 'pts_diff': -2.7592592592592595}

In [42]:
pd.DataFrame([compare_weighted_stats('Spain', 'Slovenia')])

,team_1,t1_win_chances,t1_pts_diff,team_2,t2_win_chances,t2_pts_diff,suggested_winner,pts_diff
0,Spain,0.1,-5.0,Slovenia,0.157986,-13.169271,Slovenia,-4.084635


### Wrap in a function

In [82]:
def compare_many_matches(list_of_matches_tuples:list, list_of_dates:list)-> pd.DataFrame: 
    list_suggestions = [compare_weighted_stats(a,b) for a,b in list_of_matches_tuples]
    df = pd.DataFrame(list_suggestions)
    df['date']= list_of_dates
    df = df[['date'] + list(df.columns[:-1])]
    return df


In [83]:
list_of_matches=[
                 # 27 Aug
                 ('Great Britain', 'Lithuania'),
                 ('Czechia', 'Portugal' ),
                 ('Montenegro', 'Germany'),
                 ('Latvia', 'Türkiye'),
                 ('Sweden', 'Finland'),
                 ('Serbia', 'Estonia'),
                 # 28 Aug
                 ('Georgia', 'Spain'),
                 ('Israel', 'Iceland'),
                 ('Belgium', 'France'),
                 ('Bosnia and Herzegovina', 'Cyprus'),
                 ('Greece', 'Italy'),
                 ('Slovenia', 'Poland')
                 ]

In [84]:
dates=['827']*6+['828']*6

In [85]:
df_predictions = compare_many_matches(list_of_matches_tuples=list_of_matches,
                                      list_of_dates= dates)

In [86]:
df_predictions

,date,team_1,team_2,t1_win_chances,t2_win_chances,t1_pts_diff,t2_pts_diff,suggested_winner,pts_diff
0,827,Great Britain,Lithuania,0.111111,0.937500,-6.555556,17.000000,Lithuania,11.777778
1,827,Czechia,Portugal,0.500000,0.516667,2.250000,-1.127778,Portugal,-1.688889
2,827,Montenegro,Germany,0.250000,0.909722,-3.250000,11.201389,Germany,7.225694
3,827,Latvia,Türkiye,0.491319,0.500000,0.494792,-1.875000,Türkiye,-1.184896
4,827,Sweden,Finland,0.180556,0.816667,-5.712963,14.344444,Finland,10.028704
5,827,Serbia,Estonia,1.000000,0.305556,32.200000,-12.092593,Serbia,22.146296
6,828,Georgia,Spain,0.055556,0.333333,-16.959259,1.416667,Spain,9.187963
7,828,Israel,Iceland,0.516667,0.250000,5.172222,-5.625000,Israel,5.398611
8,828,Belgium,France,0.122222,1.000000,-14.959259,7.444444,France,11.201852
9,828,Bosnia and Herzegovina,Cyprus,0.416667,0.182143,-2.027778,-27.489153,Bosnia and Herzegovina,12.730688


### Download outputs

* Table of predictions
* Table of statistics (imputed)
* Table of statistics (raw)


In [99]:
df_predictions

,date,team_1,team_2,t1_win_chances,t2_win_chances,t1_pts_diff,t2_pts_diff,suggested_winner,pts_diff
0,827,Great Britain,Lithuania,0.111111,0.937500,-6.555556,17.000000,Lithuania,11.777778
1,827,Czechia,Portugal,0.500000,0.516667,2.250000,-1.127778,Portugal,-1.688889
2,827,Montenegro,Germany,0.250000,0.909722,-3.250000,11.201389,Germany,7.225694
3,827,Latvia,Türkiye,0.491319,0.500000,0.494792,-1.875000,Türkiye,-1.184896
4,827,Sweden,Finland,0.180556,0.816667,-5.712963,14.344444,Finland,10.028704
5,827,Serbia,Estonia,1.000000,0.305556,32.200000,-12.092593,Serbia,22.146296
6,828,Georgia,Spain,0.055556,0.333333,-16.959259,1.416667,Spain,9.187963
7,828,Israel,Iceland,0.516667,0.250000,5.172222,-5.625000,Israel,5.398611
8,828,Belgium,France,0.122222,1.000000,-14.959259,7.444444,France,11.201852
9,828,Bosnia and Herzegovina,Cyprus,0.416667,0.182143,-2.027778,-27.489153,Bosnia and Herzegovina,12.730688


In [100]:
wide

,team,class_team,count_Top,count_Normal,won_mean_Top,won_mean_Normal,pts_diff_mean_Top,pts_diff_mean_Normal
0,Belgium,Normal,1.0,5.000000,0.000000,0.200000,-12.0,-15.6
1,Bosnia and Herzegovina,Normal,1.0,4.000000,0.000000,0.500000,-37.0,4.25
2,Cyprus,Normal,1.0,1.000000,0.000000,0.000000,-65.0,-40.0
3,Czechia,Normal,3.0,3.000000,0.000000,0.666667,-19.0,9.333333
4,Estonia,Normal,1.0,4.000000,0.000000,0.750000,-21.0,2.0
5,Finland,Normal,NaN,4.000000,NaN,1.000000,NaN,22.25
6,France,Top,3.0,2.000000,1.000000,1.000000,9.333333,6.5
7,Georgia,Normal,1.0,5.000000,0.000000,0.000000,-23.0,-10.6
8,Germany,Top,6.0,NaN,0.833333,NaN,5.666667,NaN
9,Great Britain,Normal,2.0,3.000000,0.000000,0.333333,-9.5,-0.666667


In [101]:
wide_imputed

,team,class_team,count_Top,count_Normal,won_mean_Top,won_mean_Normal,pts_diff_mean_Top,pts_diff_mean_Normal
0,Belgium,Normal,1.0,5.000000,0.000000,0.200000,-12.0,-15.6
1,Bosnia and Herzegovina,Normal,1.0,4.000000,0.000000,0.500000,-37.0,4.25
2,Cyprus,Normal,1.0,1.000000,0.000000,0.000000,-65.0,-40.0
3,Czechia,Normal,3.0,3.000000,0.000000,0.666667,-19.0,9.333333
4,Estonia,Normal,1.0,4.000000,0.000000,0.750000,-21.0,2.0
5,Finland,Normal,1.0,4.000000,0.000000,1.000000,-17.277778,22.25
6,France,Top,3.0,2.000000,1.000000,1.000000,9.333333,6.5
7,Georgia,Normal,1.0,5.000000,0.000000,0.000000,-23.0,-10.6
8,Germany,Top,6.0,1.000000,0.833333,1.000000,5.666667,13.96875
9,Great Britain,Normal,2.0,3.000000,0.000000,0.333333,-9.5,-0.666667


In [102]:
# Export tables and add them to dvc

repo_storage_location='C:/Users/Carlos Ivan/Documents/Projects/eurobasket_predictor/data/'
drive_location_no_versioned='G:/My Drive/ML-AI/Projects/eurobasket_predictor/data/'

for a_storage_path in [repo_storage_location, drive_location_no_versioned]:
    df_predictions.to_excel(a_storage_path + 'matches_predictions_round_1.xlsx', index=False)
    wide.to_excel(a_storage_path + 'team_performance_stats.xlsx', index=False)
    wide_imputed.to_excel(a_storage_path + 'team_performance_stats_smoothed.xlsx', index=False)